In [1]:

from omegaconf import DictConfig,OmegaConf
conf = OmegaConf.load('config/config.yaml')
pic_=conf['config']['pic_']
tracking_uri_=conf['config']['tracking_uri']

In [2]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pickle5 as pickle
from catboost import Pool
from sklearn.ensemble import StackingClassifier, VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split,StratifiedKFold

from sklearn.metrics import accuracy_score, f1_score,cohen_kappa_score,roc_auc_score,log_loss
import mlflow

In [3]:
##load_feature
fs=[]
with open('ml_output/04_05_modeling/feature_selection/fs.pickle', 'rb') as handle:
    fs_=pickle.load(handle)
    
    catboost_params=fs_['catboost-wo_artificial']
    fs_cat_wo_na=catboost_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_cat_wo_na)
    catboost_params=fs_['catboost-w_artificial']
    fs_cat_w_a=catboost_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_cat_w_a)
    
    lgbm_params=fs_['lightgbm-wo_artificial']
    fs_lgbm_wo_na=lgbm_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_lgbm_wo_na)
    lgbm_params=fs_['lightgbm-w_artificial']
    fs_lgbm_w_a=lgbm_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_lgbm_w_a)
    
    rf_params=fs_['randomforest-wo_artificial']
    fs_rf_wo_na=rf_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_rf_wo_na)
    rf_params=fs_['randomforest-wo_artificial']
    fs_rf_w_a=rf_params['params.feature_name'].replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    fs.extend(fs_rf_w_a)
    
    fs=list(dict.fromkeys(fs))

In [4]:
##Load model
model=[]
with open('ml_output/04_05_modeling/tuning/model_tuned.pickle', 'rb') as handle:
    model=pickle.load(handle)

In [5]:
##Load data
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +".csv",sep=',')
data_sample=data_sample
X=data_sample[fs]
y=data_sample[['nama_valid']]
    
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)

In [6]:

#sk_fold=StratifiedKFold(n_splits=5,shuffle=False)
experiment_name = "Modeling and Stacking"
    ## check if the experiment already exists
if not mlflow.get_experiment_by_name(experiment_name):
        mlflow.create_experiment(name=experiment_name) 
experiment = mlflow.get_experiment_by_name(experiment_name)
mlflow.set_tracking_uri(tracking_uri_)

In [7]:
def clf_estimate(clf,f_select,params):
    global valid_x, valid_y,experiment
    pred_y=clf.predict(valid_x[f_select])
    preds_proba_y=clf.predict_proba(valid_x[f_select])
    f1_micro_=f1_score(valid_y, pred_y,average='micro')
    log_loss_=log_loss(valid_y,preds_proba_y)
    roc_auc_score_=roc_auc_score(valid_y, preds_proba_y, average="weighted", multi_class="ovr")
    cohen_kappa_score_=cohen_kappa_score(valid_y, pred_y)
    with mlflow.start_run(experiment_id = experiment.experiment_id,
                          run_name=params):
        mlflow.log_metric("f1_score", np.mean(f1_micro_))
        mlflow.log_metric("log_loss", np.mean( log_loss_))
        mlflow.log_metric("roc_auc", np.mean(roc_auc_score_))
        mlflow.log_metric("cohen_kappa", np.mean(cohen_kappa_score_))
        mlflow.log_param('classifier',params)
        if params[:1]=='c':
            mlflow.catboost.log_model(clf, params)
        elif params[:1]=='l':
            mlflow.lightgbm.log_model(clf, params)
        else:
            mlflow.sklearn.log_model(clf,params)

In [ ]:
params='catboost_wo_na'
clf_estimate(model[params],fs_cat_wo_na,params)

params='catboost_w_a'
clf_estimate(model[params],fs_cat_w_a,params)

params='lightgbm_wo_na'
clf_estimate(model[params],fs_lgbm_wo_na,params)

params='lightgbm_w_a'
clf_estimate(model[params],fs_lgbm_w_a,params)

params='lightgbm_wo_na'
clf_estimate(model[params],fs_lgbm_wo_na,params)

params='randomforest_wo_na'
clf_estimate(model[params],fs_rf_wo_na,params)

cat_na=model['randomforest_w_a']
clf_estimate(model[params],fs_rf_w_a,params)

/opt/tljh/user/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
estimators_na = [('catboost_wo_na',model['catboost_wo_na']),
              ('lightgbm_wo_na',model['lightgbm_wo_na']),
              ('randomforest_wo_na',model['randomforest_wo_na'])]
clf_na = StackingClassifier(
    estimators=estimators_na, final_estimator=LogisticRegression()
    )
clf_na.fit(train_x,train_y)
clf_estimate(clf_na,fs,'stacking_wo_na')


estimators_a = [('catboost_w_a',model['catboost_w_a']),
              ('lightgbm_w_a',model['lightgbm_w_a']),
              ('randomforest_w_a',model['randomforest_w_a'])]
clf_a = StackingClassifier(
    estimators=estimators_a, final_estimator=LogisticRegression()
    )
clf_a.fit(train_x,train_y)
clf_estimate(clf_a,fs,'stacking_w_a')

eclf_na = VotingClassifier(estimators=estimators_na, voting='soft')
eclf_na.fit(train_x,train_y)
clf_estimate(eclf_na,fs,'voting_wo_na')

eclf_a = VotingClassifier(estimators=estimators_a, voting='soft')
eclf_a.fit(train_x,train_y)
clf_estimate(eclf_a,fs,'voting_w_a')

estimators = [('catboost_wo_na',model['catboost_wo_na']),
                   ('lightgbm_wo_na',model['lightgbm_wo_na']),
                   ('randomforest_wo_na',model['randomforest_wo_na']),
                   ('catboost_w_a',model['catboost_w_a']),
                   ('lightgbm_w_a',model['lightgbm_w_a']),
                   ('randomforest_w_a',model['randomforest_w_a'])]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
    )
clf.fit(train_x,train_y)
clf_estimate(clf,fs,'stacking_combined')

eclf = VotingClassifier(estimators=estimators, voting='soft')
eclf.fit(train_x,train_y)
clf_estimate(eclf,fs,'voting_combined')